# **Greedy algorithm**
This lab focuses on performing a greedy algorithm on the parametric system we saw together in the previous lab.

In [ ]:
!git clone https://github.com/fvicini/CppToPython.git
%cd CppToPython

In [ ]:
!git submodule init
!git submodule update

In [ ]:
!mkdir -p externals
%cd externals
!cmake -DINSTALL_VTK=OFF -DINSTALL_LAPACK=OFF ../gedim/3rd_party_libraries
!make -j4
%cd ..

In [ ]:
!mkdir -p release
%cd release 
!cmake -DCMAKE_PREFIX_PATH="/content/CppToPython/externals/Main_Install/eigen3;/content/CppToPython/externals/Main_Install/triangle;/content/CppToPython/externals/Main_Install/tetgen;/content/CppToPython/externals/Main_Install/googletest" ../
!make -j4 GeDiM4Py
%cd ..

In [ ]:
import numpy as np
import GeDiM4Py as gedim
from scipy.sparse.linalg import splu
import time

In [ ]:
lib = gedim.ImportLibrary("./release/GeDiM4Py.so")

config = { 'GeometricTolerance': 1.0e-8 }
gedim.Initialize(config, lib)

## The parametric version of the heat conductivity equation

Solve the following equation on square ${\Omega} = (-1, +1) \times (-1, +1)$

$$
\begin{cases}
\nabla \cdot (k_{\mu} \nabla u) = 0 & \text{in } \Omega\\
k_{\mu} \nabla u \cdot n_1 = \mu_2 & \text{in } \Gamma_{base}\\
u = 0 & \text{in } \Gamma_{top}\\
k_{\mu} \nabla u \cdot n_2 = 0 & \text{otherwise} 
\end{cases}
$$

where $k_{\mu} = \mu_1$ if $x^2 + y^2 \leq R^2$ and $k = 1$ otherwise. 
The parametric space is $\mathcal P = [0.1, 10] \times [-1,1]$.

<img src="https://drive.google.com/uc?id=1j98eKPtRy8IqsLMKkue2dINRy6yNS20j"
 style="float:center;width:50px;height:50px;" align="center">


The parameter $\boldsymbol \mu \in \mathcal P$ is physical and changes the features of the flow: 

1. $\mu_1$ the conductivity in $\Omega_1$;
2. $\mu_2$ describes the heat flux in the bottom part of the boundary.

First thing: we define two subdomains $\Omega_1$ and $\Omega_2$, such that
1. $\Omega_1$ is a disk in the origin with radius $r_0=0.5$, and
2. $\Omega_2=\Omega/\ \overline{\Omega_1}$.
3. $\Gamma_{base}$ to define where we will change the heat flux.


In [ ]:
def Heat_R():
	return 0.5


def Omega1(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values = np.ones(numPoints)
	for p in range(0, numPoints):
		if (matPoints[0,p] * matPoints[0,p] + matPoints[1,p] * matPoints[1,p]) > (Heat_R() * Heat_R() + 1.0e-16):
			values[p] = 0.
	return values.ctypes.data

def Omega2(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values = np.ones(numPoints)
	for p in range(0, numPoints):
		if (matPoints[0,p] * matPoints[0,p] + matPoints[1,p] * matPoints[1,p]) <= (Heat_R() * Heat_R() + 1.0e-16):
			values[p] = 0. 
	return values.ctypes.data

def Gamma_base(numPoints, points):
	values = np.ones(numPoints)
	return values.ctypes.data

##### needed for the inner product #####

def Domain(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values = np.ones(numPoints)
	return values.ctypes.data	

**Goal**: build the ROM space where many simulations for several parameters can be performed in a smaller amount of time. 

**Strategy**: $w(\boldsymbol  \mu)  \xrightarrow[]{\text{FOM} (\dim = \mathcal N)} w^{\mathcal N}(\boldsymbol \mu)
\xrightarrow[\lvert \lvert {w(\boldsymbol \mu) - w^\mathcal{N}(\boldsymbol  \mu)\rvert \rvert } \rightarrow 0]{\text{ROM } (\dim N)} w_N(\boldsymbol  \mu)$.

The goal can be reached by means of several techniques. 

Today we will focus on Greedy.

Building the space $\mathbb V_N \subset \mathbb V^{\mathcal N}$ and store the $\boldsymbol \mu-$independent quantities is the so called _offline phase_ (possibly costly).

Once the space is built, a fast _online phase_ occurs, where I can compute **many solutions in real-time**.

We still use the affine decomposition property. Indeed we know that our system can be written as

$$
\sum_{i=1}^{q_a} \theta_i^a(\boldsymbol \mu)a_i(u,v) = \sum_{j=0}^{q_f} \theta_j^f(\boldsymbol \mu)f_j(v),
$$

i.e., algebraic-wise

$$
\mathbb{A}(\boldsymbol \mu) = \sum_{i=1}^{q_a} \theta_i^a(\boldsymbol \mu) \mathbb{A}_i = \sum_{j=0}^{q_f} \theta_j^f(\boldsymbol \mu)\mathbf f_j = \mathbf f(\boldsymbol \mu),
$$
where $\mathbb{A}_i$ and $\mathbf f_j$ are the assembled matrices and vectors of the system.

Now, let us imagine to have already built the reduced space and have collected the basis functions $\xi_{i} \in \mathbb R^{\mathcal N}$ for $i \in \{1, \dots, N \}$ ($\mathbb V_N = \text{span}\{\xi_i\}_{i=1}^{N} $) in a basis matrix 
$$
\mathbb B = [\xi_1 \cdots \xi_N] \in \mathbb R^{\mathcal N \times N}.
$$ 

It is clear that we can recast the problem in the low-dimensional framework we built, we can pre-and-post multiply the FOM matrices for the basis matrix we have:

$$
\mathbb{A}_i^N = \mathbb B^T \mathbb{A}_i\mathbb B \quad \text{ and } \quad  \mathbf f_j^N = \mathbb B^T\mathbf f_j.  
$$

**Let us code the OFFLINE PHASE**

In [ ]:
### order of the discretization ###
order = 1

In [ ]:
%%writefile ImportMesh.csv
InputFolderPath
./Meshes/Mesh1

In [ ]:
[meshInfo, mesh] = gedim.ImportDomainMesh2D(lib)

In [ ]:
gedim.PlotMesh(mesh)

### FEM space (the High Fidelity approximation)

In [ ]:
discreteSpace = { 'Order': order, 'Type': 1, 'BoundaryConditionsType': [1, 3, 3, 2] }
[problemData, dofs, strongs] = gedim.Discretize(discreteSpace, lib)

In [ ]:
gedim.PlotDofs(mesh, dofs, strongs)

### Assemble linear system exploting affinity
We define everything that is parameter independent:
$$\mathbb{A}_i,\ i \in \{0,\dots, q_a\} \quad \mathbf{f}_j,\ j \in \{0,\dots, q_f\}.$$
Moreover, we define the matrix $\mathbb{X}$ related to the scalar product of the problem at hand.
Finally, we create the parameter dependent variable:
$$θ^a_i(\boldsymbol{\mu}),\ i \in \{0,\dots, q_a\} \quad θ^f_j(\boldsymbol{\mu}),\ j \in \{0,\dots, q_f\}$$


In [ ]:
[stiffness1, stiffnessStrong1] = gedim.AssembleStiffnessMatrix(Omega2, problemData, lib)
[stiffness2, stiffnessStrong2] = gedim.AssembleStiffnessMatrix(Omega1, problemData, lib)

weakTerm_down1 = gedim.AssembleWeakTerm(Gamma_base, 1, problemData, lib)

#### inner product X
# ||u||^2 + ||grad(u)||^2
X = stiffness1 + stiffness2  ######## semi-norm (equivalent)

### define the problem
AQH = [stiffness1, stiffness2]
fQH = [weakTerm_down1]

def thetaA(mu):
    return [1.0, mu[0]]
def thetaF(mu):
    return [mu[1]]

We will define some useful functions to perform computations:

In [ ]:
def normX(v, X):
	return np.sqrt(np.transpose(v) @ X @ v)

def ProjectSystem(AQH, fQH, B):
    AQN = []
    fQN = []
    for AH in AQH:
        AQN.append(np.copy(np.transpose(B) @ AH @ B))
    for fH in fQH:
        fQN.append(np.copy(np.transpose(B) @ fH))
    return [AQN, fQN]

def Solve_full_order(AQH, fQH, thetaA_mu, thetaF_mu):
    A = thetaA_mu[0] * AQH[0]
    f = thetaF_mu[0] * fQH[0]
    for i in range(1, len(AQH)):
        A += thetaA_mu[i] * AQH[i]
    for i in range(1, len(fQH)):
        f += thetaF_mu[i] * fQH[i]
    return gedim.LUSolver(A, f, lib)

def Solve_reduced_order(AQN, fQN, thetaA_mu, thetaF_mu):
    A = thetaA_mu[0] * AQN[0]
    f = thetaF_mu[0] * fQN[0]
    for i in range(1, len(AQN)):
        A += thetaA_mu[i] * AQN[i]
    for i in range(1, len(fQN)):
        f += thetaF_mu[i] * fQN[i]
    return np.linalg.solve(A, f)

We here define the finite parametric space $\mathcal P_{train}$, with random uniform distributed realization of $\boldsymbol \mu$.
The cardinality of $\mathcal P_{train}$ is set to $M=100$.

In [ ]:
### define the training set
M = 100
mu1_range = [0.1, 10.]
mu2_range = [-1., 1.]
P = np.array([mu1_range, mu2_range])

training_set = np.random.uniform(low=P[:, 0], high=P[:, 1], size=(M, P.shape[0]))

### POD

We recall POD algorithm. We will use it for comparisons.

To build the $N-$dimesional framework we need, we define the correlation snapshot matrix $\mathbb C \in \mathbb R^{M \times M}$ and we solve the eigenvalue problem
$
    \mathbb C \omega_n = \lambda_n \omega_n
$ for $ 1 \leq n \leq M,$ with $\lvert \lvert {\omega_n}\rvert \rvert_{\mathbb V} = 1$. 
Due to the definition of correlation matrix, we can order the all-positive eigenvalues as $\lambda_1 >\dots > \lambda_{M}> 0$ and retain the first $N$ eigenpairs $(\lambda_n, \omega_n)$ for $1 \leq n \leq N$. 

We choose $M$ and $N$ looking at the eigenvalues.
Indeed, defining as  $P_N: \mathbb V \rightarrow \mathbb V_N$ the projector from $\mathbb V$ onto $ {\mathbb V}_N$, the following relation holds:
\begin{equation}
    \sqrt{\frac{1}{M}
    \sum_{i = 1}^{M}  \lvert \lvert {u^{\mathcal N}(\boldsymbol{\mu}_{i}) - P_N(u^{\mathcal N}(\boldsymbol{\mu}_i)\rvert \rvert }_{\mathbb V}^2} = \sqrt{
    \sum_{i = N + 1}^{M}\lambda_m.}
\end{equation}
Namely, a fast decay of the eigenvalue magnitude guaratees a good representation of the high-fidelity solution with a few basis functions.

Finally, we create the basis matrix $\mathbb B$. 
There are many ways to build the bases.
We propose the following one to guarantee more stability:
$$    
\chi_n =  \sum_{m = 1}^{M} (\omega_n)_m u^{\mathcal N}(\boldsymbol{\mu}_m),  \quad \quad 1 \leq n \leq N,
$$
and $\displaystyle \xi_n = \frac{\chi_n}{\lvert \lvert \chi_n \rvert \rvert }_{\mathbb V}$.

In [ ]:
def POD(AQH, fQH, X, N_max, tol):
    #### snapshot matrix creation
    snapshot_matrix = []

    for mu in training_set:
        snapshot = Solve_full_order(AQH, fQH, thetaA(mu), thetaF(mu))
        snapshot_matrix.append(np.copy(snapshot))
    
    snapshot_matrix = np.array(snapshot_matrix) 

    ### covariance matrix
    C = snapshot_matrix @ X @ np.transpose(snapshot_matrix) ## metti inner product
    L_e, VM_e = np.linalg.eig(C)
    eigenvalues = []
    eigenvectors = []

    for i in range(len(L_e)):
        eig_real = L_e[i].real
        eig_complex = L_e[i].imag
        assert np.isclose(eig_complex, 0.)
        eigenvalues.append(eig_real)
        eigenvectors.append(VM_e[i].real)

    total_energy = sum(eigenvalues)
    retained_energy_vector = np.cumsum(eigenvalues)
    relative_retained_energy = retained_energy_vector/total_energy

    if all(flag==False for flag in relative_retained_energy >= (1.0 - tol)):
        N = N_max
    else:
        N = np.argmax(relative_retained_energy >= (1.0 - tol)) + 1

    # Create the basis function matrix
    basis_functions = []
    for n in range(N):
        eigenvector =  eigenvectors[n]
        # basis = (1/np.sqrt(M))*np.transpose(snapshot_matrix)@eigenvector 
        basis = np.transpose(snapshot_matrix) @ eigenvector
        norm = normX(basis, X)
        # norm = np.sqrt(np.transpose(basis)@basis)
        basis /= norm
        basis_functions.append(np.copy(basis))

    return [N, np.transpose(np.array(basis_functions))]

### Greedy

The **greedy generation** of the reduced basis space is an iterative procedure where at each iteration one new basis function is added and the overall precision of the basis set is improved. 
The "ideal" version of the Greedy algorithm reads as:

Given a train set $\mathcal{P}_{train}$, define $u_0 (\mu) := 0$

\begin{align}
\text{for}\ &N \in \{1, \dots , N_{max}\}:\\
&\boldsymbol{\mu}_N = \text{arg}\max_{\boldsymbol{\mu} \in \mathcal{P}_{train}} ||u^{\mathcal{N}}(\boldsymbol{\mu})-u_{N-1}(\boldsymbol{\mu})||_V\\
&S_N = S_{N−1} \cup \boldsymbol{\mu}_N\\
&\mathbb{B}_N = \mathbb{B}_{N-1} \cup \text{span}\{u_N(\boldsymbol{\mu}_N)\}
\end{align}

Possible issues are:
* $M=|\mathcal{P}_{train}|$ high fidelity solutions;
* Suboptimality (heuristic).

To overcome the first we use a sharp, inexpensive **a posteriori error bound** 
$$||u^{\mathcal{N}}(\boldsymbol{\mu})-u_{N-1}(\boldsymbol{\mu})||_V \leq Δ_N(\boldsymbol{\mu})$$
thus the algorithm becomes:

\begin{align}
\text{for}\ &N \in \{1, \dots , N_{max}\}:\\
&\boldsymbol{\mu}_N = \text{arg}\max_{\mu \in \mathcal{P}_{train}} Δ_N(\boldsymbol{\mu})\\
&S_N = S_{N−1} \cup \boldsymbol{\mu}_N\\
&\mathbb{B}_N = \mathbb{B}_{N-1} \cup \text{span}\{u_N(\boldsymbol{\mu}_N)\}
\end{align}

****
**NOTE**

POD basis is orthonormal by construction, in the greedy setting the snapshots are **not** (necessarily) orthogonal.
In order to obtain an orthonormal basis we rely on the **Gram-Schmidt
orthonormalization**, for $n > 1$:
$$z_n = u^{\mathcal{N}}(\boldsymbol{\mu}_n)-\sum_{i=0}^{N-1} (u^{\mathcal{N}}, \xi_i)_{\mathbb{X}}\xi_i$$

In [ ]:
def GramSchmidt(V, u, X):
    z = u
    if np.size(V) > 0:
        z = u - V @ (np.transpose(V) @ (X @ u))
    return z / normX(z, X)

##### Greedy #####
def Greedy(AQH, fQH, X, N_max, tol):
    N = 0
    basis_functions = []
    B = np.empty((0,0))
    deltaN = tol + 1.
    training_set_list = training_set.tolist()
    initial_muN = np.random.choice(len(training_set_list) - 1, 1)[0]
    muN = training_set_list.pop(initial_muN)
    invX = splu(X)

    print('Perfom greedy algorithm...')
    while len(training_set_list) > 0 and N < N_max and deltaN > tol:
        N = N + 1
        print('\t', N,'/', N_max, '-', '{:.16e}'.format(np.mean(deltaN)), '/', '{:.16e}'.format(np.mean(tol)))
        snapshot = Solve_full_order(AQH, fQH, thetaA(muN), thetaF(muN))
        basis_function = GramSchmidt(B, snapshot, X)
        basis_functions.append(np.copy(basis_function))
        B = np.transpose(np.array(basis_functions))
        BX = np.transpose(B) @ X @ B

        [AQN, fQN] = ProjectSystem(AQH, fQH, B)
        [Cq1q2, dq1q2, Eq1q2] = OfflineResidual(AQH, fQH, B, invX)

        counter = 0
        mu_selected_index = -1
        max_deltaN = -1.
        for mu in training_set_list:
            solN_mu = Solve_reduced_order(AQN, fQN, thetaA(mu), thetaF(mu))
            betaN_mu = InfSupConstant(mu)
            deltaN_mu = ErrorEstimate(Cq1q2, dq1q2, Eq1q2, thetaA(mu), thetaF(mu), solN_mu, betaN_mu) / normX(solN_mu, BX)
	    
            if deltaN_mu > max_deltaN:
                max_deltaN = deltaN_mu
                mu_selected_index = counter

            counter = counter + 1

        if mu_selected_index == -1:
            raise Exception('ERROR, parameter not found')

        muN = training_set_list.pop(mu_selected_index)
        deltaN = max_deltaN

    return [N, np.transpose(np.array(basis_functions))]


To compute the estimator $Δ_N(\boldsymbol{\mu})$ we rely on the error bound
$$\frac{1}{\gamma^{\mathcal{N}}(\boldsymbol{\mu})} ||r(\boldsymbol{\mu})||_{\mathbb{V}'}\leq ||e^{\mathcal{N}}(\boldsymbol{\mu})|| \leq \frac{1}{\beta^{\mathcal{N}}(\boldsymbol{\mu})} ||r(\boldsymbol{\mu})||_{\mathbb{V}'}  $$
where $e^{\mathcal{N}}(\boldsymbol{\mu}) := u^{\mathcal{N}}(\boldsymbol{\mu})-u_{N}(\boldsymbol{\mu})$ and $r(\boldsymbol{\mu}) \in \mathbb{V}'$ is the **residual** of the high-fidelity problem computed on the reduced solution, $\forall v \in \mathbb{V}$:
$$_{\mathbb{V}'} \langle r(\boldsymbol{\mu}), v  \rangle_{\mathbb{V}} = r(v; \boldsymbol{\mu}) := f(v; \boldsymbol{\mu}) - a(u_N(\boldsymbol{\mu}), v; \boldsymbol{\mu})$$

From the error bound, we define 
$$Δ_N(\boldsymbol{\mu}) := \frac{||r(\boldsymbol{\mu})||_{\mathbb{V}'}}{\beta^{\mathcal{N}}(\boldsymbol{\mu})}$$

To compute algebrically $Δ_N(\boldsymbol{\mu})$ we define the algebraic residual
$$r^{\mathcal{N}}(u_{N}; \boldsymbol{\mu}) := \mathbb{f}(\boldsymbol{\mu}) - \mathbb{A}(\boldsymbol{\mu}) \mathbb{B} u_{N}(\boldsymbol{\mu})$$
and we see from the definition that
$$\mathbb{A}(\boldsymbol{\mu}) e^{\mathcal{N}}(\boldsymbol{\mu}) = r^{\mathcal{N}}(u_{N}; \boldsymbol{\mu})$$
****
**Estimator $Δ_N(\boldsymbol{\mu})$ in $||\cdot||_2$**

Taking the $l^2$-norm on both side of the previous identity we obtain:
$$||e^{\mathcal{N}}(\boldsymbol{\mu})||_2 \leq ||\mathbb{A}^{-1}(\boldsymbol{\mu})||_2 ||r^{\mathcal{N}}(u_{N}; \boldsymbol{\mu})||_2 = \frac{1}{σ_{min}(\mathbb{A}(\boldsymbol{\mu}))} ||r^{\mathcal{N}}(u_{N}; \boldsymbol{\mu})||_2$$

****
**Estimator $Δ_N(\boldsymbol{\mu})$ in $||\cdot||_{\mathbb{X}^{-1}}$**

Similarly as before, we multiply the previous identity by $\mathbb{X}^{\frac{1}{2}}$, thus :
$$||\mathbb{X}^{\frac{1}{2}}e^{\mathcal{N}}(\boldsymbol{\mu})||_2 \leq ||\mathbb{X}^{\frac{1}{2}}\mathbb{A}^{-1}(\boldsymbol{\mu})\mathbb{X}^{\frac{1}{2}}||_2 ||\mathbb{X}^{-\frac{1}{2}}r^{\mathcal{N}}(u_{N}; \boldsymbol{\mu})||_2 = \frac{1}{σ_{min}(\mathbb{X}^{-\frac{1}{2}}\mathbb{A}(\boldsymbol{\mu})\mathbb{X}^{-\frac{1}{2}})} ||\mathbb{X}^{-\frac{1}{2}} r^{\mathcal{N}}(u_{N}; \boldsymbol{\mu})||_2$$
obtaining
$$||e^{\mathcal{N}}(\boldsymbol{\mu})||_{\mathbb{X}} \leq \frac{1}{\beta^{\mathcal{N}}(\boldsymbol{\mu})}||r^{\mathcal{N}}(u_{N}; \boldsymbol{\mu})||_{\mathbb{X}^{-1}}$$
as it is possible to show that $\beta^{\mathcal{N}}(\boldsymbol{\mu}) = σ_{min}(\mathbb{X}^{-\frac{1}{2}}\mathbb{A}(\boldsymbol{\mu})\mathbb{X}^{-\frac{1}{2}})$ if $\mathbb{A}(\boldsymbol{\mu})$ is symmetric.

****
**Offline-Online $Δ_N(\boldsymbol{\mu})$ Computation**

If the affine assumption is valid, than
\begin{align}
||r^{\mathcal{N}}(u_{N}; \boldsymbol{\mu})||_{\mathbb{X}^{-1}} = &\sum_{q_1=1}^{q_f}\sum_{q_2=1}^{q_f} \theta_{q_1}^f(\boldsymbol \mu) \theta_{q_2}^f(\boldsymbol \mu) \underbrace{\mathbf{f}^T_{q_1} \mathbb{X}^{-1} \mathbf{f}_{q_2}}_{C_{q_1, q_2}}\\
- 2 &\sum_{q_1=1}^{q_a}\sum_{q_2=1}^{q_f} \theta_{q_1}^a(\boldsymbol \mu) \theta_{q_2}^f(\boldsymbol \mu) u^T_N(\boldsymbol \mu) \underbrace{\mathbb{B}^T\mathbb{A}^T_{q_1} \mathbb{X}^{-1} \mathbf{f}_{q_2}}_{\mathbf{d}_{q_1, q_2}}\\
&\sum_{q_1=1}^{q_a}\sum_{q_2=1}^{q_a} \theta_{q_1}^a(\boldsymbol \mu) \theta_{q_2}^a(\boldsymbol \mu) u^T_N(\boldsymbol \mu) \underbrace{\mathbb{B}^T\mathbb{A}^T_{q_1} \mathbb{X}^{-1} \mathbb{A}_{q_2}\mathbb{B}}_{\mathbb{E}_{q_1, q_2}} u_N(\boldsymbol \mu)
\end{align}

In [ ]:
def OfflineResidual(AQH, fQH, B, invX):
    Cq1q2 = [] 
    dq1q2 = []
    Eq1q2 = []

    for q1 in range(0, len(AQH)):
        Z = invX.solve(AQH[q1] @ B)
        
        aqh_list = []
        for q2 in range(0, len(AQH)):
            aqh_list.append(np.copy(np.transpose(Z) @ AQH[q2] @ B))
        Eq1q2.append(aqh_list.copy())
        
        fqh_list = []
        for q2 in range(0, len(fQH)):
            fqh_list.append(np.copy(np.transpose(Z) @ fQH[q2]))
        dq1q2.append(fqh_list.copy())

    for q1 in range(0, len(fQH)):
        t = invX.solve(fQH[q1])
        
        fqh_list = []
        for q2 in range(0, len(fQH)):
            fqh_list.append(np.copy(np.transpose(t) @ fQH[q2]))
        Cq1q2.append(fqh_list.copy())

    return [Cq1q2, dq1q2, Eq1q2]
    
def InfSupConstant(mu):
    return np.min(thetaA(mu))

def ErrorEstimate(Cq1q2, dq1q2, Eq1q2, thetaA_mu, thetaF_mu, solN, beta):
    fError = 0.0
    for q1 in range(0, len(Cq1q2)):
        for q2 in range(0, len(Cq1q2[q1])):
            fError += thetaF_mu[q1] * thetaF_mu[q2] * Cq1q2[q1][q2]
    
    uError = 0.0
    for q1 in range(0, len(Eq1q2)):
        for q2 in range(0, len(Eq1q2[q1])):
            uError += thetaA_mu[q1] * thetaA_mu[q2] * np.transpose(solN) @ Eq1q2[q1][q2] @ solN
    
    fuError = 0.0
    for q1 in range(0, len(dq1q2)):
        for q2 in range(0, len(dq1q2[q1])):
            fuError += thetaA_mu[q1] * thetaF_mu[q2] * np.transpose(solN) @ dq1q2[q1][q2]
    
    deltaN_squared = fError - 2.0 * fuError + uError
    if abs(deltaN_squared) <= 1.0e-12: # protect cancellation error
        deltaN_squared = 0.0
    elif deltaN_squared < 1.0e-12:
      raise Exception('deltaN_squared is negative')
    
    return np.sqrt(deltaN_squared) / beta

### Offline Phase


In [ ]:
tol = 1.0e-7
N_max = 20

We perform now the POD as for comparison:

In [ ]:
### Compute POD
[N_POD, B_POD] = POD(AQH, fQH, X, N_max, tol)
print("N_POD", N_POD)

[AQN_POD, fQN_POD] = ProjectSystem(AQH, fQH, B_POD)

Now the greedy with the same parameters

In [ ]:
### Compute Greedy

[N_Greedy, B_Greedy] = Greedy(AQH, fQH, X, N_max, tol)
print("N_Greedy", N_Greedy)

[AQN_Greedy, fQN_Greedy] = ProjectSystem(AQH, fQH, B_Greedy)

### Online Phase

In the _online phase_ we can use all the pre-assembled quantities to generate a new solution for a new parameter. 



In [ ]:
thetaA2 = 2.
thetaf1 = 0.8

In [ ]:
def TestSingleParameter(AQH, fQH, AQN, fQN, B, mu):
    reduced_solution = Solve_reduced_order(AQN, fQN, thetaA(mu), thetaF(mu))
    full_solution = Solve_full_order(AQH, fQH, thetaA(mu), thetaF(mu))

    ###### plot #######
    proj_reduced_solution = B @ reduced_solution

    ### computing error
    error_function = full_solution - proj_reduced_solution
    error_norm_squared_component = np.transpose(error_function) @ X @ error_function
    abs_err = np.sqrt(abs(error_norm_squared_component))

    full_solution_norm_squared_component = np.transpose(full_solution) @  X @ full_solution
    rel_err = abs_err / np.sqrt(abs(full_solution_norm_squared_component))

    #gedim.PlotSolution(mesh, dofs, strongs, proj_reduced_solution, np.zeros(problemData['NumberStrongs']))
    #gedim.PlotSolution(mesh, dofs, strongs, full_solution, np.zeros(problemData['NumberStrongs']))
    
    return [rel_err, abs_err]

In [ ]:
[rel_err_POD, abs_err_POD] = TestSingleParameter(AQH, fQH, AQN_POD, fQN_POD, B_POD, [thetaA2, thetaf1])
print("SingleParameter POD relative error = ", '{:.16e}'.format(np.mean(rel_err_POD)) )
print("SingleParameter POD absolute error = ", '{:.16e}'.format(np.mean(abs_err_POD)) )

[rel_err_Greedy, abs_err_Greedy] = TestSingleParameter(AQH, fQH, AQN_Greedy, fQN_Greedy, B_Greedy, [thetaA2, thetaf1])
print("SingleParameter Gdy relative error = ", '{:.16e}'.format(np.mean(rel_err_Greedy)) )
print("SingleParameter Gdy absolute error = ", '{:.16e}'.format(np.mean(abs_err_Greedy)) )

We can now compute an error analysis over the parametric space, together with a _speed-up_ anaslysis.

The speed-up is an index that evaluated how many ROM solution I can obtain in the time of a FOM simulation.

In [ ]:
def Avg_error(AQH, fQH, AQN, fQN, B):
    ### compute avg error
    abs_err = []
    rel_err = []
    testing_set = np.random.uniform(low=P[:, 0], high=P[:, 1], size=(100, P.shape[0]))
    speed_up = []

    print("Computing error and speedup analysis...")

    for mu in testing_set:
        ##### full #####
        start_fom = time.time()
        full_solution = Solve_full_order(AQH, fQH, thetaA(mu), thetaF(mu))
        time_fom = time.time() - start_fom

        #### reduced #####
        start_rom = time.time()
        reduced_solution = Solve_reduced_order(AQN, fQN, thetaA(mu), thetaF(mu))
        time_rom = time.time() - start_rom

        speed_up.append(time_fom / time_rom)

        proj_reduced_solution = B @ reduced_solution

        ### computing error
        error_function = full_solution - proj_reduced_solution
        error_norm_squared_component = np.transpose(error_function) @ X @ error_function
        absolute_error = np.sqrt(abs(error_norm_squared_component))
        abs_err.append(absolute_error)

        full_solution_norm_squared_component = np.transpose(full_solution) @  X @ full_solution
        relative_error = absolute_error/np.sqrt(abs(full_solution_norm_squared_component))
        rel_err.append(relative_error)
    
    return [rel_err, abs_err, speed_up]

In [ ]:
[rel_err_POD, abs_err_POD, speed_up_POD] = Avg_error(AQH, fQH, AQN_POD, fQN_POD, B_POD)
print("- Average POD relative error = ", '{:.16e}'.format(np.mean(rel_err_POD)) )
print("- Average POD absolute error = ", '{:.16e}'.format(np.mean(abs_err_POD)) )
print("- Average POD speed_up       = ", '{:.16e}'.format(np.mean(speed_up_POD)))

[rel_err_Greedy, abs_err_Greedy, speed_up_Greedy] = Avg_error(AQH, fQH, AQN_Greedy, fQN_Greedy, B_Greedy)
print("- Average Gdy relative error = ", '{:.16e}'.format(np.mean(rel_err_Greedy)) )
print("- Average Gdy absolute error = ", '{:.16e}'.format(np.mean(abs_err_Greedy)) )
print("- Average Gdy speed_up       = ", '{:.16e}'.format(np.mean(speed_up_Greedy)))